In [36]:
from USTs import USTs
import datetime
from datetime import timedelta
import QuantLib as ql
import numpy as np
from typing import List
import calendar
from dateutil.relativedelta import relativedelta
import math

In [2]:
# Bill pricing
issue_date = datetime.date(2004, 1, 22)
maturity_date = datetime.date(2004, 2, 19)

bill_1 = USTs(auction_data= None, price_data=None).get_bill_discount_rate(price=99.937778,
                                                                          issue_date=issue_date,
                                                                          maturity_date=maturity_date)
bill_1

0.8

In [3]:
issue_date = datetime.date(2004, 1, 22)
maturity_date = datetime.date(2004, 2, 19)
bill_2 = USTs(auction_data= None, price_data=None).get_bill_BEYTM(price=99.937778,
                                                                          issue_date=issue_date,
                                                                          maturity_date=maturity_date)
bill_2

0.811

In [4]:
issue_date = datetime.date(1990, 6, 7)
maturity_date = datetime.date(1991, 6, 6)
bill_2 = USTs(auction_data= None, price_data=None).get_bill_BEYTM(price=92.265000,
                                                                          issue_date=issue_date,
                                                                          maturity_date=maturity_date)
bill_2

8.237

In [ ]:
issue_date = datetime.date(2025, 5, 15)
maturity_date = datetime.date(2035, 5, 15)

def adjust_for_weekend(date: datetime.date) -> datetime.date: # type: ignore
    if date.weekday() in [5, 6]:
        date = date + timedelta(days=(7 - date.weekday()))
    return date

def get_coupon_dates(issue_date, maturity_date) -> List[datetime.date]: # type: ignore
    payment_set = []
    payment_set.append(maturity_date)
    current_date = maturity_date

    while current_date > issue_date:
        current_date = maturity_date - relativedelta(months=len(payment_set) * 6)
        current_date = adjust_for_weekend(current_date)
        if current_date > issue_date:
            payment_set.append(current_date)
    payment_set.sort()
    return payment_set
def get_dates_and_cashflows(issue_date: datetime.date, maturity_date: datetime.date, coupon: float, get_days: bool = True, FV: int = 100):
    dates = get_coupon_dates(issue_date=issue_date, maturity_date=maturity_date)
    coupon_amt = coupon / 2
    if get_days:
        days = [(date - datetime.datetime.now().date()).days for date in dates]
    return_list = list()
    if get_days:
        for day in days[:-1]:
            if day > 0:
                return_list.append((day, coupon_amt))
        return_list.append((days[-1], coupon_amt + FV))
    else:
        for date in dates[:-1]:
            return_list.append((date, coupon_amt))
        return_list.append((dates[-1], coupon_amt + FV))
    return return_list

days_and_cashflows = get_dates_and_cashflows(issue_date=issue_date, maturity_date=maturity_date, coupon=4.25)

# Calculating bond price
def calculate_bond_price(days_and_cashflows: List[tuple], discount_rate: float):
    PV = 0
    DISCOUNT_RATE = discount_rate / 100
    for day, cashflow in days_and_cashflows:
        pmt_value = cashflow/((1 + DISCOUNT_RATE)**(day/183))
        PV += pmt_value
    return PV

calculate_bond_price(days_and_cashflows=days_and_cashflows, discount_rate=2.125)

def error_function(discount_rate: float, days_and_cashflows: List[tuple], actual_price: float):
    calculated_price = calculate_bond_price(days_and_cashflows=days_and_cashflows, discount_rate=discount_rate)
    error = actual_price - calculated_price
    return error

In [76]:
from scipy.optimize import newton
def get_ytm(actual_price, coupon, issue_date, maturity_date):
    days_and_cashflows = get_dates_and_cashflows(issue_date, maturity_date, coupon, get_days=True)
    guess = coupon / 2 / 100

    try:
        ytm = newton(
            func=error_function,
            x0=guess,
            args=(days_and_cashflows, actual_price)
        )
        return round(float(ytm * 2), 6)
    except RuntimeError:
        return None

get_ytm(99.28, 4.25, issue_date, maturity_date)

4.406176

In [72]:
import rateslib as rl

bond = rl.FixedRateBond(
    effective=datetime.datetime(2025, 5, 15),
    termination=datetime.datetime(2035, 5, 15),
    fixed_rate=4.250,
    spec="us_gb"  # US Government Bond
)

In [78]:
bond.ytm(price=99.28, settlement=datetime.datetime(2025, 6, 24), dirty=False)

4.339818261472328